<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_KoBERT_dial_eval_jj_ks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_jj.pt'
TESTSET_PATH = '/content/drive/MyDrive/KAIST/SEP531/ks_10000.csv'

In [16]:
!ls -l /content/drive/MyDrive/KAIST/SEP531

total 2555618
-rw-------  1 root root   1894595 Nov  8 06:21  4차년도.csv
-rw-------  1 root root   2142130 Dec  4 08:42  4_utf8.csv
-rw-------  1 root root   2167298 Dec  4 09:52  4차년도_utf8.csv
-rw-------  1 root root   2642158 Nov  8 06:21  5차년도_2차.csv
-rw-------  1 root root   2976104 Dec  4 08:41  5_2_utf8.csv
-rw-------  1 root root   2976104 Dec  4 09:52  5차년도_2차_utf8.csv
-rw-------  1 root root   1433551 Nov  8 06:21  5차년도.csv
-rw-------  1 root root   1659315 Dec  4 08:41  5_utf8.csv
-rw-------  1 root root   1678755 Dec  4 09:52  5차년도_utf8.csv
-rw-------  1 root root    737587 Dec 14 14:24  cc_10000.csv
drwx------  2 root root      4096 Dec  9 10:54  HAK_TO_HYUNG
drwx------  4 root root      4096 Nov 23 14:55  hw3_cp
drwx------ 12 root root      4096 Dec  6 14:20  hw4_cp
-rw-------  1 root root    492018 Dec 14 14:22  jj_10000.csv
-rw-------  1 root root    611318 Dec 14 14:08  jr_10000.csv
-rw-------  1 root root 368848687 Nov  8 07:19  KoBERT_emotion_f

#KoBERT 다운로드(사용)#

In [17]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-dhg2a1m3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dhg2a1m3


In [18]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [19]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [20]:
#GPU 사용
device = torch.device("cuda:0")

In [21]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [22]:
!pip install -U torchtext==0.6.0
!pip install transformers

In [23]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [24]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#KoBERT 입력 데이터로 변환(사용)#

In [25]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [26]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [28]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [29]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [30]:
import csv

texts = []
labels = []

with open(TESTSET_PATH, 'r') as f:
  csv_data = csv.reader(f, delimiter='|')
  next(csv_data)
  for i, line in enumerate(csv_data):
    text = line[0]
    label = int(line[1])
    texts.append(text)
    labels.append(label)

In [31]:
len(texts)

9999

In [32]:
y = []
p = []
def Eval():
  for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    label_idx = label
    #label_idx = r_dic.get(label)
    idx, val = predict(text)
    #print('(%s/%s) %s\n' % (val, label, text))
    if(i % 1000 == 0):
      print('[%d](%c)(%d/%d) %s\n' % (i, 'O' if (idx == label_idx) else 'X', idx, label_idx, text))
    p.append(idx)
    y.append(label_idx)

  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [33]:
Eval()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0](O)(2/2) 아~ 나는 지금 강아지 키우는데 일곱 살이거든.

[1000](O)(4/4) 나는 나도 영어 공부하고 싶어서 만나는 거잖아.

[2000](X)(5/3) 니는 진짜 의지가

[3000](O)(2/2) 근데 걔가 헤라 광고 하잖아.

[4000](O)(3/3) 그래서 그러다가 이제 고등학교 때 와서 이제 뭔가 어디를 가지 이렇게 하다가

[5000](O)(4/4) 한 사람이 우리를 끌어주고 우리는 뒤에 그냥 가만히 앉아서 구경만 하고

[6000](X)(6/4) 마이홈 고양이 버전인 거지.

[7000](X)(4/6) 내가 한 번 더 그~ 한국 돌아와서 있다가 얘랑 일 년 지나고 나서

[8000](X)(3/6) 내가 지금 고삼인데 수능을 안 쳐 그래서 할 게 없는거야.

[9000](O)(4/4) 그리고 좀만 이렇게 걸어가면은 월드컵 경기장이 있어 광주 월드컵 경기장



#F1 score

In [34]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))

accuracy 0.7424742474247424
precision1 [0.70618557 0.75689882 0.65462754 0.72061538 0.76619235 0.78083832
 0.67590028]
precision2 0.7230368934675692
precision3 0.7230368934675692
precision4 0.7424742474247424
recall1 [0.65628743 0.70158343 0.63043478 0.77242744 0.82816229 0.72767857
 0.55517634]
recall2 0.6959643260816476
recall3 0.6959643260816476
recall4 0.7424742474247424
f1_score1 [0.68032278 0.72819216 0.64230343 0.74562241 0.79597298 0.75332178
 0.60961899]
f1_score2 0.7079077913862386
f1_score3 0.7079077913862386
f1_score4 0.7424742474247423
              precision    recall  f1-score   support

           0       0.71      0.66      0.68       835
           1       0.76      0.70      0.73      1642
           2       0.65      0.63      0.64       460
           3       0.72      0.77      0.75      1516
           4       0.77      0.83      0.80      3771
           5       0.78      0.73      0.75       896
           6       0.68      0.56      0.61       879

    accurac

In [35]:
from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

print("\nLogistic Regression")
# assuming your already have a list of actual_class and predicted_class from the logistic regression classifier
lr_roc_auc_multiclass = roc_auc_score_multiclass(y, p)
print(lr_roc_auc_multiclass)


Logistic Regression
{0: 0.8157037300344749, 1: 0.8286545868685431, 2: 0.8071976827395088, 3: 0.859454319161371, 4: 0.8375718328037365, 5: 0.853787654383955, 6: 0.7647592210047303}
